In [3]:
import pymysql
import pandas as pd
import os

conn = pymysql.connect(
    host = os.environ.get("RDS_HOST"),
    user= os.environ.get("RDS_ID"),
    password= os.environ.get("RDS_PASSWORD"),
    db= os.environ.get("RDS_DB"),
    port = 3306,
)

cursor = conn.cursor()

In [2]:
def get_data_from_accuracy_mysql(tea_name='정승제'):
    df = pd.read_sql(f"SELECT * FROM pronunciation_accuracy WHERE teacher='{tea_name}'", conn)
    df2 = df.iloc[:, 7:]
    return df2

In [15]:
import plotly.graph_objects as go
import plotly.express as px

def speak_frame(tea_name):

    df = get_data_from_accuracy_mysql(tea_name)
    df_transposed = df.transpose()

    df_transposed.columns = ['Value']
    df_transposed = df_transposed.iloc[0:].reset_index(drop=False)

    fig = go.Figure(data=[go.Table(
        header=dict(values=df_transposed.columns),
        cells=dict(values=[df_transposed[col] for col in df_transposed.columns])
    )])
    fig.update_layout(
        autosize=False,
        width=500,
        title_text = f'{tea_name} 선생님 발음의 정확도',
        title_x = 0.5, title_y=0.85,
        font=dict(size=15)
    )

    # Display the figure
    fig.show()


In [16]:
speak_frame('박자영')

C:\Users\user\AppData\Local\Temp\ipykernel_22012\556109775.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 말의 공백 시각화

In [4]:
def get_data_from_speech_mysql(tea_name='정승제'):
    # MySQL 연결 생성
    #conn = pymysql.connect(**db_config)
    # # 데이터베이스에서 데이터를 가져오기 위한 SQL 쿼리
    df = pd.read_sql(f"SELECT * FROM speech_silence  WHERE teacher='{tea_name}'", conn)

    return df

In [5]:
import plotly.graph_objects as go

def silence_graph(tea_name):
    df = get_data_from_speech_mysql(tea_name)


    fig = go.Figure()
    fig.add_trace(go.Box(x=df['delta_std'], orientation='h'))

    # 그래프 크기 조절
    fig.update_layout(width=800, height=400)

    # 라벨 및 제목 레이아웃 설정
    fig.update_layout(
        xaxis=dict(title='델타의 표준편차', showticklabels=True, range=[0,4]),
        yaxis=dict(title='', showticklabels=False),
        title=dict(text = f'<{tea_name}>선생님의 휴지기간',x=0.5),
        barmode='stack',
        margin=dict(l=0,r=0,t=30, b=0)
    )
    
    
    fig.show()


In [6]:
silence_graph('박자영')

C:\Users\user\AppData\Local\Temp\ipykernel_24564\1700729404.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM speech_silence  WHERE teacher='{tea_name}'", conn)


## 강사 두명이 들어올때 시각화

In [71]:
def silence_graph(tea_name, tea_name2):
    df = get_data_from_speech_mysql(tea_name)
    df2 = get_data_from_speech_mysql(tea_name2)

    fig = go.Figure()

    fig.add_trace(go.Box(x=df2['delta_std'], orientation='h', name=tea_name2))
    fig.add_trace(go.Box(x=df['delta_std'], orientation='h', name=tea_name))
    # Add trace for the second dataframe (tea_name2)



    # 그래프 크기 조절
    fig.update_layout(width=800, height=400)

    # 라벨 및 제목 레이아웃 설정
    fig.update_layout(
        xaxis=dict(title='델타의 표준편차', showticklabels=True, range=[0.3, 4]),
        yaxis=dict(title='', showticklabels=True),
        title=dict(text=f'<{tea_name},{tea_name2}>선생님의 휴지기간 비교', x=0.5),
        barmode='stack',
        margin=dict(l=0, r=0, t=30, b=0)
    )

    # 범례 조절
    fig.update_layout(
        legend=dict(
            traceorder='reversed',
            title='',
            x=0, y=0,
            tracegroupgap=5,
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        margin=dict(
            l=0,r=0,t=30, b=0
        )
    )

    # Display the figure
    fig.show()


In [72]:
silence_graph('박자영','서지나')

C:\Users\user\AppData\Local\Temp\ipykernel_22012\1700729404.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\user\AppData\Local\Temp\ipykernel_22012\1700729404.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 침묵시간, 발화시간 비율 시각화

In [13]:

def speech_graph(tea_name):
    df = get_data_from_speech_mysql(tea_name)

    silence_time = sum(df['silence_time'])
    talking_time = sum(df['speech_time'])
    total_time = sum(df['total_time'])
    sumdata = {'침묵시간':[silence_time], '발화시간':[talking_time]}

    df = pd.DataFrame(sumdata)

    # 전체 발화시간에 대한 침묵시간과 발화시간의 비율 계산
    total_speech_time = df['발화시간'].values[0] + df['침묵시간'].values[0]
    silence_percentage = (df['침묵시간'].values[0] / total_speech_time) * 100
    speech_percentage = (df['발화시간'].values[0] / total_speech_time) * 100

    # Plotly Express를 사용하여 가로 막대 그래프 생성
    fig = px.bar(df, orientation='h')

     # 라벨 및 제목 레이아웃 설정
    fig.update_layout(
        xaxis=dict(title='', showticklabels=False),
        yaxis=dict(title='', showticklabels=False),
        title=dict(text = f'<{tea_name}> 선생님의 침묵시간 및 발화시간 비율',x=0.5),
        barmode='stack'
    )

    # 그래프 크기 조절
    fig.update_layout(width=800, height=400)

    # 각 막대에 텍스트 추가
    fig.add_annotation(x=df['침묵시간'].values[0], y=0.3, text=f'침묵시간: {silence_percentage:.2f}%',  arrowcolor='rgba(0,0,0,0)', font=dict(size=13))
    fig.add_annotation(x=df['발화시간'].values[0], y=0.3, text=f'발화시간: {speech_percentage:.2f}%', arrowcolor='rgba(0,0,0,0)', font=dict(size=13))

    fig.update_xaxes(title_text='', title_standoff=20, showticklabels=False)  # title_standoff을 조절하여 레이블의 위치 조정
    fig.update_yaxes(title_text=f'', title_standoff=0,range=[-1, 1], showticklabels=False)  # title_standoff을 조절하여 레이블의 위치 조정

    # 범례 조절
    fig.update_layout(
        legend=dict(
            traceorder='reversed',
            title='',
            x=0, y=0,
            tracegroupgap=5,
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        margin=dict(
            l=0,r=0,t=30, b=0
        )
    )

    # 그래프 표시
    fig.show()

In [14]:
speech_graph('박자영')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1700729404.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



## 선생님 두명

In [15]:
import plotly.express as px
import pandas as pd

def speech_graph(tea_name, tea_name2):
    df1 = get_data_from_speech_mysql(tea_name)
    df2 = get_data_from_speech_mysql(tea_name2)

    # Calculate total silence time, talking time, and total time for each teacher
    silence_time1 = sum(df1['silence_time'])
    talking_time1 = sum(df1['speech_time'])

    silence_time2 = sum(df2['silence_time'])
    talking_time2 = sum(df2['speech_time'])

    # Calculate ratios for each teacher
    total_time1 = talking_time1 + silence_time1
    total_time2 = talking_time2 + silence_time2

    silence_ratio1 = silence_time1 / total_time1
    speech_ratio1 = talking_time1 / total_time1

    silence_ratio2 = silence_time2 / total_time2
    speech_ratio2 = talking_time2 / total_time2

    # Create a DataFrame for the combined data
    sumdata = {'침묵 비율': [silence_ratio2, silence_ratio1], '발화 비율': [speech_ratio2, speech_ratio1]}
    df = pd.DataFrame(sumdata, index=[tea_name, tea_name2])

    # Plotly Express를 사용하여 가로 막대 그래프 생성
    fig = px.bar(df, orientation='h', labels={'index': '선생님', 'value': '비율'}, title=f'<{tea_name}, {tea_name2}> 선생님의 침묵 및 발화 비율')

    # 라벨 및 제목 레이아웃 설정
    fig.update_layout(barmode='stack')

    # 그래프 크기 조절
    fig.update_layout(width=800, height=400)

    # 각 막대에 텍스트 추가
    for idx, row in df.iterrows():
        fig.add_annotation(x=row['침묵 비율'], y=idx, text=f'침묵 비율: {row["침묵 비율"]:.2%}', arrowcolor='rgba(0,0,0,0)', font=dict(size=13))
        fig.add_annotation(x=row['발화 비율'], y=idx, text=f'발화 비율: {row["발화 비율"]:.2%}', arrowcolor='rgba(0,0,0,0)', font=dict(size=13))

    # 범례 조절
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            title='',
            x=0, y=1,
            tracegroupgap=5,
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        margin=dict(
            l=0, r=0, t=30, b=0
        )
    )

    # 막대의 너비 조절
    fig.update_layout(bargap=0.7)  # Adjust the bargap value as needed for your preference

    # 그래프 표시
    fig.show()

In [16]:
speech_graph('박자영', '서지나')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1700729404.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1700729404.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 음역대

In [17]:
def get_data_from_pitch_mysql(tea_name='정승제'):
    # MySQL 연결 생성
    #conn = pymysql.connect(**db_config)
    # # 데이터베이스에서 데이터를 가져오기 위한 SQL 쿼리
    df = pd.read_sql(f"SELECT * FROM pitch WHERE teacher='{tea_name}'", conn)

    return df

# 음역대 한명 re

In [37]:
import plotly.express as px
import numpy as np

def pitch_graph(tea_name):
  # 데이터 가져오기
  df1 = get_data_from_pitch_mysql(tea_name)
  df1_interval_pitch_median = df1.groupby(['interval_value', 'teacher'])['median_deviation'].median().reset_index()

  # Z-score를 기반으로 이상치 제거하는 함수
  def remove_outliers_zscore(df, column, threshold=3):
      z_scores = (df[column] - df[column].mean()) / df[column].std()
      df_filtered = df[np.abs(z_scores) < threshold]
      return df_filtered

  # 각각의 선생님에 대해 이상치 제거
  df1_interval_pitch_median = remove_outliers_zscore(df1_interval_pitch_median, 'median_deviation')

  # df1과 df2를 합치기
  df1_interval_pitch_median['강사'] = tea_name


  # Cohensd 함수 정의
  def cohensd(df, df2):
      sample1 = df
      sample2 = df2
      n1 = len(sample1)
      n2 = len(sample2)
      mean1 = np.mean(sample1)
      mean2 = np.mean(sample2)
      std1 = np.std(sample1, ddof=1)
      std2 = np.std(sample2, ddof=1)
      pooled_var = ((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2)
      cohensd = (mean1 - mean2) / np.sqrt(pooled_var)
      return cohensd

  # 일관된 막대 두께를 위한 bin width 지정
  bin_width = int((df1_interval_pitch_median['median_deviation'].max() - df1_interval_pitch_median['median_deviation'].min()) / 30)*4


  # Plotly Express를 사용하여 히스토그램 생성
  fig = px.histogram(df1_interval_pitch_median, x='median_deviation', color='teacher', nbins=bin_width,
                    opacity=0.7, marginal='box', labels={'median_deviation': 'Median Pitch 값'})

  # 중앙값에 대한 수직선 추가
  fig.add_shape(type='line', x0=np.median(df1_interval_pitch_median['median_deviation']),
                x1=np.median(df1_interval_pitch_median['median_deviation']),
                y0=0, y1=1, xref='x', yref='paper', line=dict(color='green', dash='dash'))



  # 중앙값 차이에 대한 화살표와 텍스트 추가
  arrow_y = 0.5  # 화살표 위치 조절
  arrow_x1 = np.median(df1_interval_pitch_median['median_deviation'])

  fig.add_annotation(x=arrow_x1, y=arrow_y, text=f'{tea_name} 중앙값: {arrow_x1}',
                    arrowhead=2, arrowcolor='black', arrowwidth=2, ax=20, ay=-50)




  # 레이아웃 업데이트
  fig.update_layout(
      title=dict(text = f'<{tea_name}> 선생님의 음역대 중간 값 분포 비교', x=0.5),
      xaxis_title='중앙값 편차',
      yaxis_title='Density',
      barmode='overlay'
  )

    # 범례 조절
  fig.update_layout(
      legend=dict(
          traceorder='normal',
          title='',
          x=0, y=1,
          tracegroupgap=5,
          bgcolor='rgba(255, 255, 255, 0.7)',
          bordercolor='rgba(255, 255, 255, 0)'
      ),
      margin=dict(
          l=0, r=0, t=30, b=0
      )
  )

  # 그래프 표시
  fig.show()

In [38]:
pitch_graph('최은진')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\4129999709.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 음역대 두명 re

---



In [35]:
import plotly.express as px
import numpy as np

def pitch_graph_com(tea_name, tea_name2):
  # 데이터 가져오기
  df1 = get_data_from_pitch_mysql(tea_name)
  df1_interval_pitch_median = df1.groupby(['interval_value', 'teacher'])['median_deviation'].median().reset_index()
  df2 = get_data_from_pitch_mysql(tea_name2)
  df2_interval_pitch_median = df2.groupby(['interval_value', 'teacher'])['median_deviation'].median().reset_index()

  # Z-score를 기반으로 이상치 제거하는 함수
  def remove_outliers_zscore(df, column, threshold=3):
      z_scores = (df[column] - df[column].mean()) / df[column].std()
      df_filtered = df[np.abs(z_scores) < threshold]
      return df_filtered

  # 각각의 선생님에 대해 이상치 제거
  df1_interval_pitch_median = remove_outliers_zscore(df1_interval_pitch_median, 'median_deviation')
  df2_interval_pitch_median = remove_outliers_zscore(df2_interval_pitch_median, 'median_deviation') #

  # df1과 df2를 합치기
  df1_interval_pitch_median['강사'] = tea_name
  df2_interval_pitch_median['강사'] = tea_name2
  combined_df = pd.concat([df1_interval_pitch_median, df2_interval_pitch_median], ignore_index=True)
  combined_df

  # Cohensd 함수 정의
  def cohensd(df, df2):
      sample1 = df
      sample2 = df2
      n1 = len(sample1)
      n2 = len(sample2)
      mean1 = np.mean(sample1)
      mean2 = np.mean(sample2)
      std1 = np.std(sample1, ddof=1)
      std2 = np.std(sample2, ddof=1)
      pooled_var = ((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2)
      cohensd = (mean1 - mean2) / np.sqrt(pooled_var)
      return cohensd

  # 효과 크기 계산
  effect_size = cohensd(df1_interval_pitch_median['median_deviation'], df2_interval_pitch_median['median_deviation'])

  # 중앙값 차이 계산
  median_difference = np.median(df1_interval_pitch_median['median_deviation']) - np.median(df2_interval_pitch_median['median_deviation'])

  # 일관된 막대 두께를 위한 bin width 지정
  bin_width = int((df1_interval_pitch_median['median_deviation'].max() - df2_interval_pitch_median['median_deviation'].min()) / 30)*4


  # Plotly Express를 사용하여 히스토그램 생성
  fig = px.histogram(combined_df, x='median_deviation', color='teacher', nbins=bin_width,
                    opacity=0.7, marginal='box', labels={'median_deviation': 'Median Pitch 값'})

  # 중앙값에 대한 수직선 추가
  fig.add_shape(type='line', x0=np.median(df1_interval_pitch_median['median_deviation']),
                x1=np.median(df1_interval_pitch_median['median_deviation']),
                y0=0, y1=1, xref='x', yref='paper', line=dict(color='green', dash='dash'))

  fig.add_shape(type='line', x0=np.median(df2_interval_pitch_median['median_deviation']),
                x1=np.median(df2_interval_pitch_median['median_deviation']),
                y0=0, y1=1, xref='x', yref='paper', line=dict(color='orange', dash='dash'))

  # 중앙값 차이에 대한 화살표와 텍스트 추가
  arrow_y = 0.5  # 화살표 위치 조절
  arrow_x1 = np.median(df1_interval_pitch_median['median_deviation'])
  arrow_x2 = np.median(df2_interval_pitch_median['median_deviation'])
  arrow_x_diff = arrow_x1 + (arrow_x2 - arrow_x1) / 2

  fig.add_annotation(x=arrow_x1, y=arrow_y, text=f'{tea_name} 중앙값: {arrow_x1}',
                    arrowhead=2, arrowcolor='black', arrowwidth=2, ax=20, ay=-50)

  fig.add_annotation(x=arrow_x2, y=arrow_y, text=f'{tea_name2} 중앙값: {arrow_x2}',
                    arrowhead=2, arrowcolor='black', arrowwidth=2, ax=-20, ay=-100)

  # 중앙값 차이에 대한 화살표와 텍스트 추가
  arrow_y_diff = 20  # 화살표 위치 조절
  fig.add_shape(type='line', x0=arrow_x1, x1=arrow_x2, y0=arrow_y_diff, y1=arrow_y_diff,
                line=dict(color='black', width=2, dash='dot'))

  fig.add_annotation(x=arrow_x_diff, y=arrow_y_diff, text=f'중앙값 차이: {median_difference:.2f}', ax=-20, ay=-10)



  # 레이아웃 업데이트
  fig.update_layout(
      title=dict(text = f'<{tea_name},{tea_name2}> 선생님의 음역대 중간 값 분포 비교 (효과 크기: {effect_size:.2f})', x=0.5),
      xaxis_title='중앙값 편차',
      yaxis_title='Density',
      barmode='overlay'
  )

    # 범례 조절
  fig.update_layout(
      legend=dict(
          traceorder='normal',
          title='',
          x=0, y=1,
          tracegroupgap=5,
          bgcolor='rgba(255, 255, 255, 0.7)',
          bordercolor='rgba(255, 255, 255, 0)'
      ),
      margin=dict(
          l=0, r=0, t=30, b=0
      )
  )

  # 그래프 표시
  fig.show()

In [36]:
pitch_graph_com('최은진', '정유빈')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\4129999709.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\user\AppData\Local\Temp\ipykernel_28544\4129999709.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 수사학 & 존댓말

In [21]:
def context_df(tea_name):
    # 선생님의 데이터를 불러와서 특정 컬럼 선택
    df = pd.read_sql(f"SELECT * FROM context WHERE teacher='{tea_name}'", conn)
    df_rds = df.iloc[:, 6:]
    list_row = []

    # 각 의도별로 반말과 존댓말 개수(합계) 계산
    for category in df_rds.columns.str.split('_').str[0].unique():
        col_list = ['id', 'datetime', 'subject', 'minor',	'teacher', 'class', 'lecture']
        if category in col_list:
          pass
        else:
          row = {
              '의도': category,
              '반말': df[f'{category}_il'].sum(),
              '존댓말': df[f'{category}_fl'].sum()
          }
          list_row.append(row)

    # 결과 데이터프레임 생성
    result_df_test = pd.DataFrame(list_row)

    # 각 의도의 반말과 존댓말 비율 계산
    result_df_test['반말_비율'] = result_df_test['반말'] / (result_df_test['반말'] + result_df_test['존댓말'])
    result_df_test['존댓말_비율'] = result_df_test['존댓말'] / (result_df_test['반말'] + result_df_test['존댓말'])

    # 의도에 대한 한글 라벨 매핑
    type_mapping = {'statement': '설명',
                    'question': '질문',
                    'command': '지시어',
                    'rq': '수사학적 질문',
                    'fragment': '불완전 문장',
                    'rc': '수사학적 명령'}
    result_df_test['의도'] = result_df_test['의도'].map(type_mapping)
    result_df_test.dropna(inplace=True)

    return result_df_test

In [22]:
def context_visualization(tea_name):
    result_df_test = context_df(tea_name)

    # 그래프 생성
    fig = go.Figure()

    # 반말과 존댓말에 대한 막대 그래프 생성
    fig.add_trace(go.Bar(x=result_df_test['의도'], y=result_df_test['반말'], name='반말'))
    fig.add_trace(go.Bar(x=result_df_test['의도'], y=result_df_test['존댓말'], name='존댓말'))

    # 라벨 및 제목 추가
    fig.update_layout(
        xaxis=dict(title='의도'),
        yaxis=dict(title='비율', showticklabels=False),
        title=dict(text = f'<{tea_name}> 선생님의 의도에 따른 반말 존댓말 비율',x=0.5),
        barmode='stack'
    )

    # 전체 비율 텍스트 표시
    total_bars = result_df_test["반말"].sum() + result_df_test["존댓말"].sum()

    # 그래프 상단에 비율 텍스트 표시
    for idx, row in result_df_test.iterrows():
        bm, jm, total = row['반말_비율'], row['존댓말_비율'], row['반말'] + row['존댓말']
        ratio_text = f'반말 : {bm:.1%} <br> 존댓말 :  {jm:.1%}<br>'
        fig.add_annotation(
            x=row['의도'],
            y=total,
            text=f'문장 비율 : {round(total / total_bars*100, 2)}% <br> {ratio_text}',
            showarrow=True,
            arrowhead=4,
            ax=0,
            ay=-40,
            font=dict(size=12)
        )

    max_value = result_df_test[['반말', '존댓말']].sum(axis=1).max()
    fig.update_yaxes(range=[0, 1.3 * max_value])

    fig.update_layout(
        legend=dict(
            traceorder='reversed',
            # title='범례',
            x=0, y=1,
            tracegroupgap=5,
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        margin=dict(
            l=0,r=0,t=30, b=0
        )
    )

    fig.show()


In [23]:
context_visualization('정승제')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1297867846.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 수사학 및 존댓말 2명

In [24]:
import plotly.graph_objects as go
def context_com(tea_name, tea_name2):

  df=context_df(tea_name)
  df['의도'] = df['의도'] + f'_{tea_name}'
  df['반말_scale'] = df['반말']/(df['반말'].sum() + df['존댓말'].sum())
  df['존댓말_scale'] = df['존댓말']/(df['반말'].sum() + df['존댓말'].sum())
  df['전체 반말 비율'] = df['반말_scale']/df['반말_scale'].sum()
  df['전체 존댓말 비율'] = df['존댓말_scale']/df['존댓말_scale'].sum()

  df2=context_df(tea_name2)
  df2['의도'] = df2['의도'] + f'_{tea_name2}'
  df2['반말_scale'] = df2['반말']/(df2['반말'].sum() + df2['존댓말'].sum())
  df2['존댓말_scale'] = df2['존댓말']/(df2['반말'].sum() + df2['존댓말'].sum())
  df2['전체 반말 비율'] = df2['반말_scale']/df2['반말_scale'].sum()
  df2['전체 존댓말 비율'] = df2['존댓말_scale']/df2['존댓말_scale'].sum()

  result_df_test = pd.concat([df, df2], ignore_index=True)


  # x축 순서 재정의
  desired_order = [f'지시어_{tea_name}', f'지시어_{tea_name2}', f'불완전 문장_{tea_name}', f'불완전 문장_{tea_name2}',
                  f'질문_{tea_name}', f'질문_{tea_name2}', f'수사학적 명령_{tea_name}', f'수사학적 명령_{tea_name2}',
                  f'수사학적 질문_{tea_name}', f'수사학적 질문_{tea_name2}', f'설명_{tea_name}', f'설명_{tea_name2}']

  # 그래프 생성
  fig = go.Figure()


  # 의도의 끝이 'tea_name'로 끝나는 것들에 대한 누적 막대 그래프
  subset_df1 = result_df_test[result_df_test['의도'].str.endswith(tea_name)]

  fig.add_trace(go.Bar(x=subset_df1['의도'], y=subset_df1['반말_scale'], name=f'반말_{tea_name}', marker_color='#FF6C6C'))
  fig.add_trace(go.Bar(x=subset_df1['의도'], y=subset_df1['존댓말_scale'], name=f'존댓말_{tea_name}', marker_color='#3D44FF'))
  total_bars_t1 = subset_df1["반말"].sum() + subset_df1["존댓말"].sum()

  # 의도의 끝이 'tea_name2'로 끝나는 것들에 대한 누적 막대 그래프
  subset_df2 = result_df_test[result_df_test['의도'].str.endswith(tea_name2)]

  fig.add_trace(go.Bar(x=subset_df2['의도'], y=subset_df2['반말_scale'], name=f'반말_{tea_name2}', marker_color='#FFAAAA'))
  fig.add_trace(go.Bar(x=subset_df2['의도'], y=subset_df2['존댓말_scale'], name=f'존댓말_{tea_name2}', marker_color='#5188FF'))
  total_bars_t2 = subset_df2["반말"].sum() + subset_df2["존댓말"].sum()

  # 라벨 및 제목 추가
  fig.update_layout(
      xaxis=dict(title='의도', categoryorder='array', categoryarray=desired_order),
      yaxis=dict(title='비율'),
      title=dict(text=f'<{tea_name}, {tea_name2}> 선생님의 의도에 따른 반말 존댓말 비율', x=0.5),
      barmode='stack'
  )

  total_bars_t1 = subset_df1["반말"].sum() + subset_df1["존댓말"].sum()
  total_bars_t2 = subset_df2["반말"].sum() + subset_df2["존댓말"].sum()

  # 그래프 상단에 비율 텍스트 표시
  for idx, row in result_df_test.iterrows():
      if row['의도'].endswith(tea_name):
          bm, jm, total, total2 = row['반말_비율'], row['존댓말_비율'], row['반말'] + row['존댓말'], row['반말_scale'] + row['존댓말_scale']
          ratio_text = f'존댓말 :  {jm:.1%} <br> 반말 : {bm:.1%}<br>'
          fig.add_annotation(
              x=row['의도'],
              y=total2,
              text=f'문장 비율 : {round(total / total_bars_t1*100, 2)}% <br> {ratio_text}',
              showarrow=True,
              arrowhead=4,
              ax=0,
              ay=-40,
              font=dict(size=12)
          )
      elif row['의도'].endswith(tea_name2):
          bm, jm, total, total2 = row['반말_비율'], row['존댓말_비율'], row['반말'] + row['존댓말'], row['반말_scale'] + row['존댓말_scale']
          ratio_text = f'존댓말 :  {jm:.1%} <br> 반말 : {bm:.1%}<br>'
          fig.add_annotation(
              x=row['의도'],
              y=total2,
              text=f'문장 비율 : {round(total / total_bars_t2*100, 2)}% <br> {ratio_text}',
              showarrow=True,
              arrowhead=4,
              ax=0,
              ay=-40,
              font=dict(size=12)
          )

  fig.update_yaxes(title_text=f'문장 비율', title_standoff=10,showticklabels=True)

  fig.update_layout(
      legend=dict(
          traceorder='reversed',
          x=0,y=1,
          tracegroupgap=5,
          bgcolor='rgba(255, 255, 255, 0.7)',
          bordercolor='rgba(255, 255, 255, 0)'
      ),
      margin=dict(
          l=0, r=0, t=30, b=0
      )
  )

  line_positions = [
      (desired_order.index(f'지시어_{tea_name2}') + desired_order.index(f'불완전 문장_{tea_name}')) / 2,
      (desired_order.index(f'불완전 문장_{tea_name2}') + desired_order.index(f'질문_{tea_name}')) / 2,
      (desired_order.index(f'질문_{tea_name2}') + desired_order.index(f'수사학적 명령_{tea_name}')) / 2,
      (desired_order.index(f'수사학적 명령_{tea_name2}') + desired_order.index(f'수사학적 질문_{tea_name}')) / 2,
      (desired_order.index(f'수사학적 질문_{tea_name2}') + desired_order.index(f'설명_{tea_name}')) / 2,
  ]

  # 각 세로선 추가
  for position in line_positions:
      fig.add_shape(
          type='line',
          x0=position,
          x1=position,
          y0=0,
          y1=1,
          line=dict(color='gray', width=1, dash='dot')
      )

  fig.show()

In [25]:
context_com('정승제', '서영란')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1297867846.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1297867846.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 반말 존댓말 비율 2명

In [26]:
def formal_com(tea_name, tea_name2):
  df1 = context_df(tea_name)
  df2 = context_df(tea_name2)

  # Calculate total silence time, talking time, and total time for each teacher
  informal_df1 = sum(df1['반말'])
  formal_df1 = sum(df1['존댓말'])

  informal_df2 = sum(df2['반말'])
  formal_df2 = sum(df2['존댓말'])

  # Calculate ratios for each teacher
  total_time1 = formal_df1 + informal_df1
  total_time2 = formal_df2 + informal_df2

  informal_ratio1 = informal_df1 / total_time1
  formal_ratio1 = formal_df1 / total_time1

  informa_ratio2 = informal_df2 / total_time2
  formal_ratio2 = formal_df2 / total_time2

  # Create a DataFrame for the combined data
  sumdata = {'반말 비율': [informa_ratio2, informal_ratio1], '존댓말 비율': [formal_ratio2, formal_ratio1]}
  df = pd.DataFrame(sumdata, index=[tea_name2, tea_name])

  # Plotly Express를 사용하여 가로 막대 그래프 생성
  fig = px.bar(df, orientation='h', labels={'index': '선생님', 'value': '비율'})

  # 레이아웃 업데이트
  fig.update_layout(
      title=dict(text = f'<{tea_name}, {tea_name2}> 선생님의 반말, 존댓말 비율', x=0.5),
      barmode='stack',
      width=800, height=300)

  # 각 막대에 텍스트 추가
  for idx, row in df.iterrows():
      fig.add_annotation(x=0, y=idx, text=f'반말 비율: {row["반말 비율"]:.2%}', arrowcolor='rgba(0,0,0,0)', font=dict(size=13))
      fig.add_annotation(x=1, y=idx, text=f'존댓말 비율: {row["존댓말 비율"]:.2%}', arrowcolor='rgba(0,0,0,0)', font=dict(size=13))

  # 범례 조절
  fig.update_layout(
      legend=dict(
          traceorder='normal',
          title='',
          x=0.4, y=0.5,
          tracegroupgap=5,
          bgcolor='rgba(255, 255, 255, 0.7)',
          bordercolor='rgba(255, 255, 255, 0)'
      ),
      margin=dict(
          l=0, r=0, t=30, b=0
      )
  )

  fig.update_xaxes(title_text='', title_standoff=20, showticklabels=False)

  # 막대의 너비 조절
  fig.update_layout(bargap=0.7)  # Adjust the bargap value as needed for your preference

  # 그래프 표시
  fig.show()

In [27]:
formal_com('정승제', '정유빈')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1297867846.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\user\AppData\Local\Temp\ipykernel_28544\1297867846.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 언어 빠르기 한명

In [28]:
def get_data_from_speed_mysql(tea_name='정승제'):
    # MySQL 연결 생성
    #conn = pymysql.connect(**db_config)
    # # 데이터베이스에서 데이터를 가져오기 위한 SQL 쿼리
    df = pd.read_sql(f"SELECT * FROM speech_speed WHERE teacher='{tea_name}'", conn)

    return df

In [31]:
import plotly.express as px
import numpy as np
def speed_visualization(tea_name):
  combined_df_2 = get_data_from_speed_mysql(tea_name)

  # 중복값 제거 및 정렬
  unique_data = combined_df_2[['scaled_percent', 'speed_median']].drop_duplicates().sort_values(by='scaled_percent')

  # 1% 간격으로 잘라서 구간별 중앙값 계산
  interval = 1
  max_percent = unique_data['scaled_percent'].max()
  bins = np.arange(0, max_percent + interval, interval)
  unique_data['scaled_percent_bin'] = pd.cut(unique_data['scaled_percent'], bins=bins, include_lowest=True)
  grouped_data = unique_data.groupby('scaled_percent_bin').median()

  # 범주형 데이터를 수치형으로 변환
  grouped_data['scaled_percent'] = grouped_data.index.categories.mid

  # Plot using Plotly
  fig = px.line(grouped_data, x='scaled_percent', y='speed_median', labels={'scaled_percent': '시간 구간 (%)', 'speed_median': '말 속도 중앙값'} )

  # Rotate x-axis labels
  fig.update_layout(title=dict(text = f'<{tea_name}> 선생님의 말 속도의 중앙값 분포', x=0.5),
                    xaxis=dict(tickangle=45),
                    title_font=dict(size=20))


  # Show the plot
  fig.show()

In [32]:
speed_visualization('정승제')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\2318525088.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



## 말속도 두명

In [33]:
import plotly.express as px
def speed_com(tea_name, tea_name2):
  combined_df_2 = get_data_from_speed_mysql('정승제')
  combined_df_3 = get_data_from_speed_mysql('서영란')


  # 중복값 제거 및 정렬 (강사1)
  unique_data_2 = combined_df_2[['scaled_percent', 'speed_median']].drop_duplicates().sort_values(by='scaled_percent')

  # 중복값 제거 및 정렬 (강사2)
  unique_data_3 = combined_df_3[['scaled_percent', 'speed_median']].drop_duplicates().sort_values(by='scaled_percent')

  # 1% 간격으로 잘라서 구간별 중앙값 계산 (강사1)
  interval = 1
  max_percent_2 = unique_data_2['scaled_percent'].max()
  bins_2 = np.arange(0, max_percent_2 + interval, interval)
  unique_data_2['scaled_percent_bin'] = pd.cut(unique_data_2['scaled_percent'], bins=bins_2, include_lowest=True)
  grouped_data_2 = unique_data_2.groupby('scaled_percent_bin').median()

  # 1% 간격으로 잘라서 구간별 중앙값 계산 (강사2)
  max_percent_3 = unique_data_3['scaled_percent'].max()
  bins_3 = np.arange(0, max_percent_3 + interval, interval)
  unique_data_3['scaled_percent_Bin'] = pd.cut(unique_data_3['scaled_percent'], bins=bins_3, include_lowest=True)
  grouped_data_3 = unique_data_3.groupby('scaled_percent_Bin').median()

  # 범주형 데이터를 수치형으로 변환 (강사1)
  grouped_data_2['scaled_percent'] = grouped_data_2.index.categories.mid

  # 범주형 데이터를 수치형으로 변환 (강사2)
  grouped_data_3['scaled_percent'] = grouped_data_3.index.categories.mid


  # Create figure and scatter plots
  fig = px.scatter(grouped_data_2, x='scaled_percent', y='speed_median', labels={'scaled_percent': '시간 구간 (%)', 'speed_median': '말 속도 중앙값'} )

  # Add line plots for 강사1 and 강사2
  fig.add_scatter(x=grouped_data_2['scaled_percent'], y=grouped_data_2['speed_median'], mode='lines', name=tea_name)
  fig.add_scatter(x=grouped_data_3['scaled_percent'], y=grouped_data_3['speed_median'], mode='lines', name=tea_name2)

  # Set layout options
  fig.update_layout(title=dict(text = f'<{tea_name}, {tea_name2}> 선생님의 말 속도의 중앙값 분포 비교', x=0.5),
                    xaxis=dict(tickangle=45),
                    xaxis_title='시간 구간 (%)',
                    yaxis_title='말 속도 중앙값',
                    legend=dict(x=0, y=1, traceorder='normal'),
                    title_font=dict(size=20))


  # Show the plot
  fig.show()

In [34]:
speed_com('정승제', '서영란')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\2318525088.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\user\AppData\Local\Temp\ipykernel_28544\2318525088.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# 농담

In [39]:
def get_data_from_mysql(tea_name='정승제'):

    # # # SQL 쿼리를 실행하고 결과를 데이터프레임으로 가져오기
    df = pd.read_sql(f"SELECT * FROM nongdam where teacher = '{tea_name}'", conn)
    
    return df

In [40]:
import plotly.graph_objects as go

def visualization_nongdam ( tea_name):

    df_nongdam = get_data_from_mysql(tea_name)
    df_nongdam_re = df_nongdam.iloc[:, 7:].sum()
    total_len = df_nongdam_re['total_len']
    target_len = df_nongdam_re['target_len']

    labels = ['강의 관련 문장', '관련없는 문장']
    values = [total_len - target_len, target_len]
    pull = [0, 0.1]

    trace = go.Pie(labels=labels, values=values, textinfo='label+percent', pull=pull)

    fig = go.Figure(data=trace)

    # 레이아웃 설정
    fig.update_layout(
        title=f'<{tea_name}>강의와 상관없는 단어 비율',
        showlegend=True,
        height=500,  # 그림의 높이 설정
        width=700,   # 그림의 너비 설정
    )

    return fig.show()

In [42]:
visualization_nongdam('정승제')

C:\Users\user\AppData\Local\Temp\ipykernel_28544\2084156629.py:4: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

